# Bayesian sentiment analysis: Yelp, IMDB, and BernoulliNB

In [189]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [190]:
yelp = pd.read_csv('../../_Datasets/yelp_labelled.txt', delimiter='\t', header=None)
yelp.columns = ['Review', 'Outcome']
yelp.head()

,Review,Outcome
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [191]:
# identify all caps reviews
yelp['allcaps'] = yelp.Review.str.isupper()

# remove punctuation and make lowercase
yelp['Review'] = yelp.Review.str.replace(r'[^a-zA-Z\d\s:]', '')
yelp['Review'] = yelp.Review.str.lower()
yelp['Review'] = yelp.Review.str.strip()
yelp['Review_length'] = yelp.Review.str.len()

In [192]:
negative_words = np.unique(yelp.Review[yelp.Outcome == 0].str.cat(sep=' ').split())
positive_words = np.unique(yelp.Review[yelp.Outcome == 1].str.cat(sep=' ').split())

diff = np.setdiff1d(ar1=pd.Series(negative_words).value_counts().keys().ravel(), 
                    ar2=pd.Series(positive_words).value_counts().keys().ravel())
diff

array(['1', '10', '1199', '12', '15', '15lb', '17', '1979', '30', '30s',
       '34ths', '35', '400', '40min', '45', '785', '90', 'accountant',
       'ache', 'acknowledged', 'actual', 'ahead', 'airline', 'ala',
       'albondigas', 'allergy', 'alone', 'although', 'angry',
       'anticipated', 'anymore', 'anytime', 'anyways', 'apart',
       'apologize', 'apology', 'appalling', 'apparently', 'appealing',
       'appetite', 'appetizer', 'apple', 'arent', 'arepas', 'arrives',
       'arriving', 'article', 'asked', 'asking', 'atrocious', 'attached',
       'attack', 'attention', 'attitudes', 'average', 'avocado', 'avoid',
       'avoided', 'awful', 'awkward', 'awkwardly', 'bad', 'ball',
       'bamboo', 'banana', 'bare', 'barely', 'based', 'basically',
       'batch', 'batter', 'beensteppedinandtrackedeverywhere', 'begin',
       'behind', 'bellagio', 'below', 'besides', 'between', 'bigger',
       'biggest', 'binge', 'bird', 'bisque', 'bitches', 'bites', 'blah',
       'blame', 'bland',

In [193]:
words = ['insults', 'rude', 'stupid', 'tasteless', 'underwhelming', 'yucky', 'bland', 'blandest',
         'boring', 'hair', 'inconsiderate', 'terrible', 'horrible', 'overcooked', 'ok', 'poop', 
         'gross', 'grossed', 'humiliated', 'lukewarm', 'mediocre', 'negligent', 'missing', 'overpriced',
         'tasteless', 'undercooked', 'underwhelming', 'unprofessional', 'zero']

for word in words:
    yelp[word] = yelp.Review.str.contains(r'\b' + word, regex=True)

In [194]:
sns.heatmap(yelp[words].corr())

In [195]:
yelp.head(1)

,Review,Outcome,allcaps,Review_length,insults,rude,stupid,tasteless,underwhelming,yucky,...,grossed,humiliated,lukewarm,mediocre,negligent,missing,overpriced,undercooked,unprofessional,zero
0,wow loved this place,1,False,20,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Naive Bayes model: creating object and applying to another dataset

In [196]:
from sklearn.naive_bayes import BernoulliNB
from sklearn import preprocessing

bern = BernoulliNB()

In [197]:
min_max = preprocessing.MinMaxScaler()
length_scaled = min_max.fit_transform(yelp.Review_length.values.reshape(-1, 1))
yelp['Review_length_scaled'] = length_scaled
words.append('Review_length_scaled')
bern.fit(yelp[words], yelp.Outcome)

F:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [198]:
imdb = pd.read_csv('../../_Datasets/imdb_labelled.txt', delimiter='\t', header=None)
imdb.columns = ['Review', 'Outcome']

# identify all caps reviews
imdb['allcaps'] = imdb.Review.str.isupper()

# remove punctuation and make lowercase
imdb['Review'] = imdb.Review.str.replace(r'[^a-zA-Z\d\s:]', '')
imdb['Review'] = imdb.Review.str.lower()
imdb['Review'] = imdb.Review.str.strip()
imdb['Review_length'] = imdb.Review.str.len()

words = ['insults', 'rude', 'stupid', 'tasteless', 'underwhelming', 'yucky', 'bland', 'blandest',
         'boring', 'hair', 'inconsiderate', 'terrible', 'horrible', 'overcooked', 'ok', 'poop', 
         'gross', 'grossed', 'humiliated', 'lukewarm', 'mediocre', 'negligent', 'missing', 'overpriced',
         'tasteless', 'undercooked', 'underwhelming', 'unprofessional', 'zero']

for word in words:
    imdb[word] = imdb.Review.str.contains(r'\b' + word, regex=True)
    
length_scaled = min_max.fit_transform(imdb.Review_length.values.reshape(-1, 1))
imdb['Review_length_scaled'] = length_scaled

imdb_pred = bern.predict(imdb.drop(columns=['Review', 'Outcome']))

F:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [199]:
pd.Series(imdb.Outcome == imdb_pred).value_counts()

False    386
True     362
Name: Outcome, dtype: int64

#### Application in another context

The application in another context was surprisingly about half. 

The selected word pool for negative food reviews has nearly no application to movie reviews, and thus it has overfitted the training data and yielded us poor results - BUT the results are much better than I expected. Perhaps review length helped with this task.